# Natural Gas Powerplants and Environmental Justice - Nationwide and in Maryland

### In this notebook, I create sub-datasets for tracts within 5km, 10km and 20km geographic radiuses for natural gas locations on a national level and Maryland level. I create datasets with the same geographic radiuses for a list of potential new power plant locations in Maryland (sites currently or formerly hosting industrial facilities).

In [93]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [95]:
%%R

require('tidyverse')
require('DescTools')

## Creating sub-datasets for natural gas locations and their nearby tracts on a national-level and Maryland-level

In [171]:
df = pd.read_csv('eji_plants_one_row_per_geoid.csv')
df.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,GEOID_2020,COUNTY_x,StateDesc,STATEABBR,LOCATION,E_TOTPOP,M_TOTPOP,E_DAYPOP,SPL_EJI,RPL_EJI,SPL_SER,RPL_SER,SPL_EJI_CBM,RPL_EJI_CBM,E_MINRTY,EPL_MINRTY,SPL_SVM_DOM1,RPL_SVM_DOM1,E_POV200,EPL_POV200,E_NOHSDP,EPL_NOHSDP,E_UNEMP,EPL_UNEMP,E_RENTER,EPL_RENTER,E_HOUBDN,EPL_HOUBDN,E_UNINSUR,EPL_UNINSUR,E_NOINT,EPL_NOINT,SPL_SVM_DOM2,RPL_SVM_DOM2,E_AGE65,EPL_AGE65,E_AGE17,EPL_AGE17,E_DISABL,EPL_DISABL,E_LIMENG,EPL_LIMENG,SPL_SVM_DOM3,RPL_SVM_DOM3,E_MOBILE,EPL_MOBILE,E_GROUPQ,EPL_GROUPQ,SPL_SVM_DOM4,RPL_SVM_DOM4,SPL_SVM,RPL_SVM,E_OZONE,EPL_OZONE,E_PM,EPL_PM,E_DSLPM,EPL_DSLPM,E_TOTCR,EPL_TOTCR,SPL_EBM_DOM1,RPL_EBM_DOM1,E_NPL,EPL_NPL,E_TRI,EPL_TRI,E_TSD,EPL_TSD,E_RMP,EPL_RMP,E_COAL,EPL_COAL,E_LEAD,EPL_LEAD,SPL_EBM_DOM2,RPL_EBM_DOM2,E_PARK,EPL_PARK,E_HOUAGE,EPL_HOUAGE,E_WLKIND,EPL_WLKIND,SPL_EBM_DOM3,RPL_EBM_DOM3,E_RAIL,EPL_RAIL,E_ROAD,EPL_ROAD,E_AIRPRT,EPL_AIRPRT,SPL_EBM_DOM4,RPL_EBM_DOM4,E_IMPWTR,EPL_IMPWTR,F_IMPWTR,SPL_EBM_DOM5,RPL_EBM_DOM5,SPL_EBM,RPL_EBM,E_ASTHMA,EPL_ASTHMA,F_ASTHMA,E_CANCER,EPL_CANCER,F_CANCER,E_CHD,EPL_CHD,F_CHD,E_MHLTH,EPL_MHLTH,F_MHLTH,E_DIABETES,EPL_DIABETES,F_DIABETES,F_HVM,RPL_HVM,E_NEHD,EPL_NEHD,SPL_CBM_DOM1,RPL_CBM_DOM1,E_BURN,EPL_BURN,E_SMOKE,EPL_SMOKE,SPL_CBM_DOM2,RPL_CBM_DOM2,E_CFLD,EPL_CFLD,F_CFLD,E_DRGT,EPL_DRGT,F_DRGT,E_HRCN,EPL_HRCN,F_HRCN,E_RFLD,EPL_RFLD,F_RFLD,E_SWND,EPL_SWND,F_SWND,E_TRND,EPL_TRND,F_TRND,SPL_CBM_DOM3,RPL_CBM_DOM3,SPL_CBM,RPL_CBM,AFAM,E_AFAM,HISP,E_HISP,ASIAN,E_ASIAN,AIAN,E_AIAN,NHPI,E_NHPI,TWOMORE,E_TWOMORE,OTHERRACE,E_OTHERRACE,Tribe_PCT_Tract,Tribe_Names,Tribe_Flag,Unnamed: 0.1,Unnamed: 0,plant_name,Plant Code,State,Sector Name,Prime Movers,Fuel Types,Primary Technology,withdrawal_volume_million_gallons,discharge_volume_million_gallons,consumption_volume_million_gallons,energy_category,map_link,longitude,latitude,STATE,COUNTY_y,TRACT,contains_energy_source,Technology_Category,Category_Number,fossil_fuels,renewables,energy_category_numeric,IntPtLat,IntPtLon,Band,distance_to_plant,num_energy_sources,plant_names
0,1.0,1.0,20100.0,140000US01001020100,1.001020e+09,1.001020e+09,Autauga County,Alabama,AL,Census Tract 201; Autauga County; Alabama,1865.0,368.0,943.0,1.3991,0.5685,0.7991,0.3536,1.5356,0.3476,23.4,0.3833,0.3833,0.3833,25.6836,0.4529,16.2,0.7660,2.6,0.2443,25.9,0.4369,17.9,0.2697,8.2,0.5842,8.1,0.3859,3.1399,0.4179,19.5,0.6666,24.0,0.6504,20.1,0.8586,0.0,0.0000,2.1756,0.6563,12.6,0.8240,0.0,0.0000,0.8240,0.5067,6.5228,0.4541,0.091,0.3188,0.0,0.0,0.1692,0.3622,40.0,0.9248,1.6058,0.5437,0.0,0.0,32.6935,0.5965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5965,0.2675,84.1618,0.6725,56.1,0.5058,6.3333,0.7379,1.9162,0.7429,74.0860,0.6896,51.2604,0.3653,0.0,0.0,1.0549,0.4931,0.0,0.0,0.0,0.0,0.0,5.1734,0.3450,11.0,0.6239,0.0,8.6,0.5721,0.0,8.4,0.7441,1.0,20.0,0.8042,1.0,13.8,0.6922,1.0,3.0,0.6,8.6,0.2039,0.2039,0.2039,0.0,0.0,4.5,0.1270,0.1270,0.1262,0.0,0.0,1.0,26.0909,0.6637,9.0,0.076,0.5691,9.0,1.0833,0.244,9.0,1.205,0.3395,9.0,0.0000,0.0000,0.0,1.8163,0.5025,2.1472,0.1365,208.0,11.2,81.0,4.3,45.0,2.4,0.0,0.0,0.0,0.0,103.0,5.5,0.0,0.0,0.0,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0.0,32.476823,-86.486599,4,14.394846,0,NaN
1,1.0,1.0,20200.0,140000US01001020200,1.001020e+09,1.001020e+09,Autauga County,Alabama,AL,Census Tract 202; Autauga County; Alabama,1861.0,396.0,4081.0,1.2619,0.4907,0.8619,0.4049,1.3951,0.2725,63.8,0.7571,0.7571,0.7571,26.2432,0.4654,8.7,0.5041,5.5,0.5945,21.1,0.3489,17.5,0.2551,7.8,0.5605,21.5,0.8530,3.5815,0.5222,15.7,0.4633,9.9,0.0591,20.4,0.8662,0.0,0.0000,1.3886,0.1199,1.2,0.5585,9.9,0.9497,1.5082,0.8877,7.2354,0.5811,0.091,0.3188,0.0,0.0,0.1769,0.3847,40.0,0.9248,1.6283,0.5518,0.0,0.0,50.9214,0.6639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6639,0.2923,100.0000,0.5514,68.2,0.6431,8.5833,0.5168,1.7113,0.5722,77.8517,0.7019,10.2446,0.1662,0.0,0.0,0.8681,0.4027,0.0,0.0,0.0,0.0,0.0,4.8716,0.2808,10.7,0.5367,0.0,6.5,0.3101,0.0,6.5,0.4235,0.0,19.0,0.7245,1.0,14.7,0.7671,1.0,2.0,0.4,8.

In [172]:
df.loc[(df['Band'] == 1) & (df['distance_to_plant'].isna()), 'distance_to_plant'] = 0

In [173]:
df['within_1_km'] = (df['Band'] == 1).astype(int)
df['within_5_km'] = df['Band'].isin([1, 2]).astype(int)
df['within_10_km'] = df['Band'].isin([1, 2, 3]).astype(int)
df['within_20_km'] = df['Band'].isin([1, 2, 3, 4]).astype(int)

In [174]:
df['natural_gas'] = df['Category_Number'] == 2

In [175]:
df_within_5_km = df[df['within_5_km'] == 1].copy()
df_within_10_km = df[df['within_10_km'] == 1].copy()
df_within_20_km = df[df['within_20_km'] == 1].copy()

In [176]:
df_within_5_km.to_csv("tracts_within_5_km_plant.csv", index=False)
df_within_10_km.to_csv("tracts_within_10_km_plant.csv", index=False)
df_within_20_km.to_csv("tracts_within_20_km_plant.csv", index=False)

In [177]:
pd.set_option('display.max_columns', None)

In [178]:
# with pd.option_context('display.max_rows', None):
#     display(df_within_5_km[df_within_5_km['StateDesc'] == 'Maryland'])

In [179]:
md_within_5_km = df_within_5_km[df_within_5_km['StateDesc'] == 'Maryland']

In [180]:
md_within_10_km = df_within_10_km[df_within_10_km['StateDesc'] == 'Maryland']

In [181]:
md_within_20_km = df_within_20_km[df_within_20_km['StateDesc'] == 'Maryland']

In [182]:
md_within_5_km.to_csv("md_tracts_within_5_km_plant.csv", index=False)
md_within_10_km.to_csv("md_tracts_within_10_km_plant.csv", index=False)
md_within_20_km.to_csv("md_tracts_within_20_km_plant.csv", index=False)

## Creating sub-datasets for proposed locations in Maryland

In [1]:
df = pd.read_csv('md_proposed_plants_one_row_per_geoid.csv')
df.head()

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


NameError: name 'pd' is not defined

In [ ]:
df.loc[(df['Band'] == 1) & (df['distance_to_plant'].isna()), 'distance_to_plant'] = 0

In [187]:
df['within_1_km'] = (df['Band'] == 1).astype(int)
df['within_5_km'] = df['Band'].isin([1, 2]).astype(int)
df['within_10_km'] = df['Band'].isin([1, 2, 3]).astype(int)
df['within_20_km'] = df['Band'].isin([1, 2, 3, 4]).astype(int)

In [188]:
df_within_5_km = df[df['within_5_km'] == 1].copy()
df_within_10_km = df[df['within_10_km'] == 1].copy()
df_within_20_km = df[df['within_20_km'] == 1].copy()

In [189]:
df_within_5_km.to_csv("tracts_within_5_km_proposed_plant.csv", index=False)
df_within_10_km.to_csv("tracts_within_10_km_proposed_plant.csv", index=False)
df_within_20_km.to_csv("tracts_within_20_km_proposed_plant.csv", index=False)